# ===========================================
# 1_data_exploration.ipynb
# Analyse der E-Mail-Texte
# ===========================================

# Zelle 1: Bibliotheken importieren
import os
import pandas as pd
import matplotlib.pyplot as plt
import re
import nltk
from nltk.tokenize import word_tokenize

# Für inline Plots (falls nicht automatisch aktiviert)
%matplotlib inline

# Stelle sicher, dass die benötigten NLTK-Daten vorhanden sind
nltk.download('punkt')

# ===========================================
# Zelle 2: Daten einlesen
# ===========================================
# Beispiel: Wir nehmen an, dass es eine CSV-Datei "emails_raw.csv" in data/raw/ gibt.
# Die CSV enthält beispielsweise die Spalten: "subject", "body", "style" (formell/informell).

RAW_DATA_DIR = "../data/raw"
data_file = os.path.join(RAW_DATA_DIR, "emails_raw.csv")

df = pd.read_csv(data_file)
print("Anzahl der E-Mails:", len(df))
display(df.head())

# ===========================================
# Zelle 3: Überblick über Spalten & fehlende Werte
# ===========================================
print("\nDataFrame Info:")
df.info()

print("\nFehlende Werte pro Spalte:")
print(df.isnull().sum())

# ===========================================
# Zelle 4: Basisstatistiken der E-Mail-Texte
# ===========================================
# Wir analysieren die Länge der Betreffzeilen und der E-Mail-Bodies.
df["subject_length"] = df["subject"].astype(str).apply(len)
df["body_length"] = df["body"].astype(str).apply(len)

print("\nStatistiken Betreffzeilen (subject):")
display(df["subject_length"].describe())

print("\nStatistiken E-Mail-Inhalte (body):")
display(df["body_length"].describe())

# ===========================================
# Zelle 5: Visualisierung der Textlängen
# ===========================================
plt.figure(figsize=(10, 4))
plt.hist(df["subject_length"], bins=30, color="steelblue", alpha=0.8)
plt.xlabel("Länge der Betreffzeilen (Zeichen)")
plt.ylabel("Anzahl E-Mails")
plt.title("Verteilung der Länge der Betreffzeilen")
plt.show()

plt.figure(figsize=(10, 4))
plt.hist(df["body_length"], bins=50, color="seagreen", alpha=0.8)
plt.xlabel("Länge der E-Mail-Inhalte (Zeichen)")
plt.ylabel("Anzahl E-Mails")
plt.title("Verteilung der Länge der E-Mail-Inhalte")
plt.show()

# ===========================================
# Zelle 6: Tokenisierung und häufige Wörter
# ===========================================
def clean_text(text):
    """Grundlegende Bereinigung: Entferne HTML-Tags und überflüssige Leerzeichen, konvertiere in Kleinbuchstaben."""
    if not isinstance(text, str):
        text = str(text)
    text = re.sub(r"<.*?>", "", text)
    text = re.sub(r"\s+", " ", text)
    return text.strip().lower()

# Wende die Bereinigung auf den E-Mail-Body an
df["clean_body"] = df["body"].apply(clean_text)

# Tokenisierung des bereinigten E-Mail-Bodys
df["body_tokens"] = df["clean_body"].apply(word_tokenize)

# Erstelle einen Counter für häufige Wörter
from collections import Counter
all_tokens = [token for tokens in df["body_tokens"] for token in tokens]
token_counts = Counter(all_tokens)

print("\nTop 10 häufigste Wörter im E-Mail-Body:")
print(token_counts.most_common(10))

# Balkendiagramm der 10 häufigsten Wörter
tokens, freqs = zip(*token_counts.most_common(10))
plt.figure(figsize=(8, 4))
plt.bar(tokens, freqs, color="mediumpurple")
plt.xlabel("Wörter")
plt.ylabel("Häufigkeit")
plt.title("Top 10 häufigste Wörter im E-Mail-Body")
plt.xticks(rotation=45)
plt.show()

# ===========================================
# Zelle 7: Analyse nach Stil (formell vs. informell)
# ===========================================
# Falls eine Spalte "style" vorhanden ist, die den E-Mail-Stil angibt.
if "style" in df.columns:
    print("\nStatistiken nach Stil:")
    for style in df["style"].unique():
        subset = df[df["style"] == style]
        print(f"\nStil: {style}")
        print("Betrefflänge:")
        display(subset["subject_length"].describe())
        print("Bodylänge:")
        display(subset["body_length"].describe())
else:
    print("\nKeine 'style'-Spalte gefunden; überspringe Stil-Analyse.")

# ===========================================
# Zelle 8: Fazit & Ausblick
# ===========================================
print("""
Fazit:
- Die Daten enthalten E-Mails mit variabler Länge in Betreff und Inhalt.
- Es gibt signifikante Unterschiede in der Länge und Wortwahl.
- Erste Tokenisierungs- und Wortfrequenzanalysen geben Aufschluss über häufige Begriffe.
Nächste Schritte:
- Weitere Bereinigung und Tokenisierung verfeinern.
- Aufteilung in Trainings-, Validierungs- und Test-Datensätze (siehe Notebook 2_data_preprocessing.ipynb).
- Entwicklung eines Modells zur E-Mail-/Textvorschlagsgenerierung (MailAssist).
""")
